# Autograde a students' solution

**These instructions are currently incomplete**.

Let's say that students have turned in their notebooks with a special naming convention of `Lastname/Problem N.ipynb`, where `N` is the problem number and `Lastname` is a directory named with the student's last name. In this example, we have a few submitted assignments:

* [Bitdiddle/Problem 1.ipynb](grade_example/submitted/Bitdiddle/Problem 1.ipynb)
* [Bitdiddle/Problem 2.ipynb](grade_example/submitted/Bitdiddle/Problem 2.ipynb)
* [Hacker/Problem 1.ipynb](grade_example/submitted/Hacker/Problem 1.ipynb)
* [Hacker/Problem 2.ipynb](grade_example/submitted/Hacker/Problem 2.ipynb)

In [1]:
cd grade_example

/Users/jhamrick/project/tools/nbgrader/docs/user_guide/grade_example


In [2]:
ls -R

nbgrader_autograde_config.py  nbgrader_formgrade_config.py  submitted/

./submitted:
Bitdiddle/ Hacker/

./submitted/Bitdiddle:
Problem 1.ipynb  Problem 2.ipynb

./submitted/Hacker:
Problem 1.ipynb  Problem 2.ipynb


## Initialize the database

Before we can actually start grading, we need to actually record who the students are. We can do this using the API provided by nbgrader, which provides access to a MongoDB to store information about students and their grades:

In [3]:
# create a connection to the db using the nbgrader API
from nbgrader.api import Gradebook
gb = Gradebook("sqlite:////tmp/nbgrader_example.db")

In [4]:
# create some students and add them to the database
gb.add_student("Bitdiddle", first_name="Ben", last_name="Bitdiddle")
gb.add_student("Hacker", first_name="Alyssa", last_name="Hacker")
gb.add_student("Reasoner", first_name="Louis", last_name="Reasoner")
gb.students

[Bitdiddle, Hacker, Reasoner]

Note that the assignment should also already be in the database. An example of how to add it is given in [3 - Releasing assignments.ipynb](3 - Releasing assignments.ipynb#Recording-grade-cells).

## Run the autograder

Now that we've set up the database, we can grade the assignments.

First, we specify the configuration file for `nbgrader autograde`:

In [5]:
!cat nbgrader_autograde_config.py

c = get_config()

import os
c.AutogradeApp.notebooks = ['submitted']
c.AutogradeApp.output_dir = os.path.join(os.getcwd(), 'autograded')
c.AutogradeApp.recursive = True
c.FindStudentID.regexp = r"submitted/(?P<student_id>.+)/.*.ipynb"
c.SaveAutoGrades.assignment_id = "Problem Set 1"
c.SaveAutoGrades.db_url = "sqlite:////tmp/nbgrader_example.db"

# Only necessary if --overwrite-cells is given
c.OverwriteGradeCells.assignment_id = "Problem Set 1"
c.OverwriteGradeCells.db_url = "sqlite:////tmp/nbgrader_example.db"


This config file says that we should convert the files in the `submitted` directory, and save the autograded files to a new directory called `autograded`. We will do this *recursively*, so that the autograder will look into the directories with the students' names. We additionally specify a regular expression which tells the grader how to determine the student ID. In this case, the ID is just their last name. Finally, we tell the autograder what assignment these notebooks are a part of, and what database the grades should be saved into.

Now, to grade the assignments, all we have to do is:

In [6]:
!nbgrader autograde

[AutogradeApp] Using existing profile dir: u'/Users/jhamrick/.nbgrader/profile_default'
[AutogradeApp] Directory tree prefix: submitted/
[AutogradeApp] Changing to directory: submitted/Bitdiddle
[AutogradeApp] Converting notebook Problem 1.ipynb to notebook
[AutogradeApp] Support files will be in Problem 1_files/


[AutogradeApp] Student ID: Bitdiddle


[AutogradeApp] Executing notebook with kernel: python3


[AutogradeApp] Writing output to directory: autograded/Bitdiddle
[AutogradeApp] Writing 26743 bytes to /Users/jhamrick/project/tools/nbgrader/docs/user_guide/grade_example/autograded/Bitdiddle/Problem 1.ipynb
[AutogradeApp] Changing to directory: submitted/Bitdiddle
[AutogradeApp] Converting notebook Problem 2.ipynb to notebook
[AutogradeApp] Support files will be in Problem 2_files/
[AutogradeApp] Student ID: Bitdiddle


[AutogradeApp] Executing notebook with kernel: python3


...

[AutogradeApp] Executing notebook with kernel: python3


[AutogradeApp] Writing output to directory: autograded/Hacker
[AutogradeApp] Writing 24718 bytes to /Users/jhamrick/project/tools/nbgrader/docs/user_guide/grade_example/autograded/Hacker/Problem 1.ipynb
[AutogradeApp] Changing to directory: submitted/Hacker
[AutogradeApp] Converting notebook Problem 2.ipynb to notebook
[AutogradeApp] Support files will be in Problem 2_files/
[AutogradeApp] Student ID: Hacker


[AutogradeApp] Executing notebook with kernel: python3


[AutogradeApp] Writing output to directory: autograded/Hacker


[AutogradeApp] Writing 2819 bytes to /Users/jhamrick/project/tools/nbgrader/docs/user_guide/grade_example/autograded/Hacker/Problem 2.ipynb


Now if we look at the `autograded` directory, we see:

* [Bitdiddle/Problem 1.ipynb](grade_example/autograded/Bitdiddle/Problem 1.ipynb)
* [Bitdiddle/Problem 2.ipynb](grade_example/autograded/Bitdiddle/Problem 2.ipynb)
* [Hacker/Problem 1.ipynb](grade_example/autograded/Hacker/Problem 1.ipynb)
* [Hacker/Problem 2.ipynb](grade_example/autograded/Hacker/Problem 2.ipynb)

In [7]:
ls -R autograded

Bitdiddle/ Hacker/

autograded/Bitdiddle:
Problem 1.ipynb  Problem 2.ipynb

autograded/Hacker:
Problem 1.ipynb  Problem 2.ipynb


## Replacing grade cells

If we [released the assignment with `--save-cells`](3 - Releasing assignments.ipynb#Recording-grade-cells) (i.e., we saved the grade cells into the database, then we can ensure the grade cells are the same as when the assignment was released by passing the `--overwrite-cells` flag to `nbgrader autograde`:

In [8]:
!nbgrader autograde --overwrite-cells

[AutogradeApp] Using existing profile dir: u'/Users/jhamrick/.nbgrader/profile_default'
[AutogradeApp] Directory tree prefix: submitted/
[AutogradeApp] Changing to directory: submitted/Bitdiddle
[AutogradeApp] Converting notebook Problem 1.ipynb to notebook
[AutogradeApp] Support files will be in Problem 1_files/


[AutogradeApp] Student ID: Bitdiddle
[AutogradeApp] WARNING | Checksum for grade cell correct_squares has changed!


[AutogradeApp] WARNING | Checksum for grade cell correct_sum_of_squares has changed!
[AutogradeApp] WARNING | Checksum for grade cell plot_axis_limits has changed!
[AutogradeApp] WARNING | Checksum for grade cell plot_data has changed!


[AutogradeApp] Executing notebook with kernel: python3


[AutogradeApp] Writing output to directory: autograded/Bitdiddle
[AutogradeApp] Writing 52565 bytes to /Users/jhamrick/project/tools/nbgrader/docs/user_guide/grade_example/autograded/Bitdiddle/Problem 1.ipynb
[AutogradeApp] Changing to directory: submitted/Bitdiddle
[AutogradeApp] Converting notebook Problem 2.ipynb to notebook
[AutogradeApp] Support files will be in Problem 2_files/
[AutogradeApp] Student ID: Bitdiddle


...

[AutogradeApp] Student ID: Hacker


[AutogradeApp] Executing notebook with kernel: python3


[AutogradeApp] Writing output to directory: autograded/Hacker
[AutogradeApp] Writing 24712 bytes to /Users/jhamrick/project/tools/nbgrader/docs/user_guide/grade_example/autograded/Hacker/Problem 1.ipynb
[AutogradeApp] Changing to directory: submitted/Hacker
[AutogradeApp] Converting notebook Problem 2.ipynb to notebook
[AutogradeApp] Support files will be in Problem 2_files/
[AutogradeApp] Student ID: Hacker


[AutogradeApp] Executing notebook with kernel: python3


[AutogradeApp] Writing output to directory: autograded/Hacker
[AutogradeApp] Writing 2819 bytes to /Users/jhamrick/project/tools/nbgrader/docs/user_guide/grade_example/autograded/Hacker/Problem 2.ipynb


Notice that for Ben Bitdiddle's submission for `Problem 1`, there are some cells that changed! If we look at his submission, we'll see that he actually commented out some of the test cell code, perhaps in an attempt to "trick" the grader. However, because we had saved the checksums and cell source in our database, we were able to overwrite his changes and grade his assignment with the correct tests